In [17]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer 
from transformers.generation import GenerationConfig
torch.manual_seed(1234)

from IPython.display import Latex, Audio

In [116]:
from tqdm.auto import tqdm

In [2]:
import pandas as pd

In [3]:
DEVICE = torch.device('cuda:1')

In [4]:
lm_head_state_dict = torch.load('ckpts/asr-sentence/version_0/lm_head_state_dict.pth', weights_only=False, map_location=DEVICE)

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer

import torch


tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-Audio", trust_remote_code=True)

# use bf16
# model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-Audio", device_map="auto", trust_remote_code=True, bf16=True).eval()
# use fp16
# model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-Audio", device_map="auto", trust_remote_code=True, fp16=True).eval()
# use cpu only
# model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-Audio", device_map="cpu", trust_remote_code=True).eval()
# use cuda device
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-Audio", device_map=DEVICE, trust_remote_code=True).eval()

eos_token_id = tokenizer('<|endoftext|>',return_tensors='pt').input_ids[0]

audio_start_id: 155163, audio_end_id: 155164, audio_pad_id: 151851.


The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Try importing flash-attention for faster inference...


Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

In [6]:
model.lm_head.load_state_dict(lm_head_state_dict)

<All keys matched successfully>

In [69]:
_ = model.eval()

In [7]:
test =pd.read_csv('test_ENG.csv')
test.head()

,speaker_id,language,formula_id,is_tts,pronunciation,whisper_transcription,latex,audio_path
0,0,eng,4178,0,mu minus nu hat.,Mu hat minus Nu hat.,\hat \mu-\hat \nu,/workspace-SR006.nfs2/shares/SR006.nfs2/Nikita...
1,0,eng,4231,0,mu tilde is defined as the set of mu one plus ...,MuTiled is defined as the set of Mu 1 plus Mu...,\tilde { \mu } : = \ { \mu_1 + \tilde { \mu } ...,/workspace-SR006.nfs2/shares/SR006.nfs2/Nikita...
2,0,eng,4321,0,N sub G of s sub 1 union N sub G of s sub 2 se...,"NG union NG set- open brace C1, C2 close brace",N_G ( s_1 ) \cup N_G ( s_2 ) \setminus \ { s_1...,/workspace-SR006.nfs2/shares/SR006.nfs2/Nikita...
3,0,eng,4374,0,"N, open parenthesis, 1 sub Q transpose, comma,...",n 1 sub q transpose comma 4 sub i sub n close...,"\mathcal { N } \left ( { 1_Q^ \top,4 { I_n } }...",/workspace-SR006.nfs2/shares/SR006.nfs2/Nikita...
4,0,eng,4507,0,Negative zeta to the power of negative one-hal...,negative zeta to the power of negative one ha...,"\mathfrak { C } = [ -\zeta^ { -1/2 } , \zeta^ ...",/workspace-SR006.nfs2/shares/SR006.nfs2/Nikita...


In [109]:
N = 101

In [110]:
# audio_url = "../2646.wav"
# audio_url = "../4763.wav"
audio_url = test.iloc[N]['audio_path']
true_latex = test.iloc[N]['latex']
sp_prompt = "<|startofanalysis|><|en|><|caption|><|en|><|notimestamps|><|wo_itn|>"
query = f"<audio>{audio_url}</audio>{sp_prompt}"
audio_info = tokenizer.process_audio(query)

In [111]:
inputs = tokenizer(query, return_tensors='pt', audio_info=audio_info)

In [121]:
gen_params = {
    "max_new_tokens": 50,
    "repetition_penalty": 1.0,
    "remove_invalid_values": True,
    "eos_token_id": eos_token_id,
    "forced_eos_token_id": eos_token_id,
    "use_cache": True,
    "no_repeat_ngram_size": 4,
    # "num_return_sequences": 1,
    }

In [113]:
inputs = inputs.to(model.device)
pred = model.generate(**inputs, **gen_params, audio_info=audio_info)
response = tokenizer.decode(pred.cpu()[0], skip_special_tokens=False, audio_info=audio_info)
print(response)
wo_ind = response.find('<|wo_itn|>')
response = response[wo_ind:].replace('<|endoftext|>', '').replace('<|wo_itn|>', '')
print(response)

/home/jovyan/.mlspace/envs/skripkin/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/jovyan/.mlspace/envs/skripkin/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:623: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
/home/jovyan/.mlspace/envs/skripkin/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:649: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


<audio>/workspace-SR006.nfs2/shares/SR006.nfs2/Nikita/speech2latex/TagMePools/eng/pool4_eng/6aba0f6b-abcf-4045-b529-f3f905474093/6695.wav</audio><|startofanalysis|><|en|><|caption|><|en|><|notimestamps|><|wo_itn|>A \mathbf{v} \in \mathbf{T}_\mathbf{u} \mathbf{n} \to \mathbf{x} \in S^2 \to \lambda \mathbf{i} \in T<|endoftext|>
A \mathbf{v} \in \mathbf{T}_\mathbf{u} \mathbf{n} \to \mathbf{x} \in S^2 \to \lambda \mathbf{i} \in T


In [114]:
Latex(f'${response}$')

<IPython.core.display.Latex object>

In [115]:
Latex(f'${true_latex}$')

<IPython.core.display.Latex object>

In [ ]:
outputs = []
for i in tqdm(range(len(test))):
    audio_url = test.iloc[N]['audio_path']
    true_latex = test.iloc[N]['latex']
    sp_prompt = "<|startofanalysis|><|en|><|caption|><|en|><|notimestamps|><|wo_itn|>"
    query = f"<audio>{audio_url}</audio>{sp_prompt}"
    audio_info = tokenizer.process_audio(query)
    inputs = tokenizer(query, return_tensors='pt', audio_info=audio_info)
    inputs = inputs.to(model.device)
    pred = model.generate(**inputs, **gen_params, audio_info=audio_info)
    response = tokenizer.decode(pred.cpu()[0], skip_special_tokens=False, audio_info=audio_info)
    # print(response)
    wo_ind = response.find('<|wo_itn|>')
    response = response[wo_ind:].replace('<|endoftext|>', '').replace('<|wo_itn|>', '')
    # print(response)

    outputs.append({
        'pred': response,
        'true_latex': true_latex
    })

  0%|          | 0/2594 [00:00<?, ?it/s]

In [ ]:
torch.save(outputs, 'outputs_lm_head_trained_1_epoch.pth')

In [1]:
print()

In [125]:
outputs

[{'pred': 'A \\mathbf{u} \\rightarrow \\mathbf{x} \\rightarrow |\\mathbf{A} \\mathbf {u} | \\lambda',
  'true_latex': 'A_\\mu \\rightarrow A_\\mu + \\partial_\\mu \\lambda'},
 {'pred': 'H^2^ : H^2^ \\to \\lambda^ { 2} \\cdot H^ { 4}',
  'true_latex': 'A_\\mu \\rightarrow A_\\mu + \\partial_\\mu \\lambda'},
 {'pred': 'In G^T (a^T) (A^T) = a (A)^T',
  'true_latex': 'A_\\mu \\rightarrow A_\\mu + \\partial_\\mu \\lambda'},
 {'pred': 'H^T (A \\cdot \\vec{u}) = A^T \\cdot \\mathbf{u} + d \\mathbf {u}',
  'true_latex': 'A_\\mu \\rightarrow A_\\mu + \\partial_\\mu \\lambda'},
 {'pred': '\\\\phi_E^ { \\mu_E^ { - \\phi_E^2 } }',
  'true_latex': 'A_\\mu \\rightarrow A_\\mu + \\partial_\\mu \\lambda'},
 {'pred': '\\int \\frac{\\partial \\mathbf{A}}{\\partial \\mu} \\mathbf{x} \\cdot \\nabla \\mathbf { \\mu } \\, d\\mathbf{x }',
  'true_latex': 'A_\\mu \\rightarrow A_\\mu + \\partial_\\mu \\lambda'},
 {'pred': 'H is a g^d^ h^d^ \\lambda',
  'true_latex': 'A_\\mu \\rightarrow A_\\mu + \\partial_\\mu